In [34]:
from imblearn.over_sampling import SMOTE, RandomOverSampler

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

In [67]:
path = '../../data/Change-Level-Prediction-Data-20191107T052353Z-001/Change-Level-Prediction-Data/ICSE-2016-PROMISE DefectData/'
train_fname = 'ant-1.5.csv'
test_fname = 'ant-1.6.csv'

In [68]:
# Get the data
train_df = pd.read_csv(path + train_fname)
test_df = pd.read_csv(path + test_fname)

In [69]:
train_df.head()

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.5,org.apache.tools.ant.taskdefs.War,9,6,0,9,34,28,0,9,6,0.812500,166,1.00,0,0.950617,0.400000,4,9,17.222222,2,1.000,1
1,ant,1.5,org.apache.tools.ant.taskdefs.Copydir,8,4,0,7,37,4,0,7,7,0.714286,277,1.00,0,0.917647,0.468750,4,4,32.875000,5,1.375,0
2,ant,1.5,org.apache.tools.ant.taskdefs.LoadProperties,4,3,0,8,39,0,0,8,4,0.333333,200,1.00,0,0.925000,0.500000,1,1,48.500000,1,0.750,0
3,ant,1.5,org.apache.tools.ant.taskdefs.cvslib.ChangeLog...,15,3,0,16,77,69,0,16,10,0.826531,504,1.00,0,0.725490,0.233333,1,1,32.133333,6,1.400,0
4,ant,1.5,org.apache.tools.tar.TarOutputStream,15,3,0,3,37,41,1,2,14,0.696429,437,0.75,1,0.454545,0.311111,1,2,27.333333,1,0.800,0


In [70]:
# Remove unnecessary columns
train_df.drop(['name', 'version', 'name.1'], inplace=True, axis=1)
test_df.drop(['name', 'version', 'name.1'], inplace=True, axis=1)

In [71]:
# Get the X and y for train and test sets
x_train = train_df.drop('bug', axis=1)
y_train = train_df['bug']

x_test = test_df.drop('bug', axis=1)
y_test = test_df['bug']

In [72]:
np.unique(y_train), np.unique(y_test)

(array([0, 1, 2]), array([ 0,  1,  2,  3,  4,  5,  6,  8, 10]))

In [61]:
# Purge the weird rows
idx = np.where(y_train < 2)[0]
x_train = np.array(x_train)[idx]
y_train = y_train[idx]

idx = np.where(y_test < 2)[0]
x_test = np.array(x_test)[idx]
y_test = y_test[idx]

In [46]:
# Alternate purge method
y_train[y_train > 1] = 1
y_test[y_test > 1] = 1

/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
# Purge method #3
y_train[y_train > 1] = 0
y_test[y_test > 1] = 0

/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [74]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((293, 20), (293,), (351, 20), (351,))

In [75]:
sampler = RandomOverSampler()

In [76]:
x_train, y_train = sampler.fit_resample(x_train, y_train)

In [50]:
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

/Users/ryedida/Library/Python/3.7/lib/python/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
preds = clf.predict(x_test)
f1_score(y_test, preds)

0.28333333333333327

Having tried both the purging methods and the RandomForest classifier, neither are close. With the first method, it yields about 12% F1; with the other, it yields about 28%.

Let's try with a different classifier

In [53]:
from sklearn.naive_bayes import GaussianNB

In [54]:
nb = GaussianNB()
nb.fit(x_train, y_train)

preds = nb.predict(x_test)
f1_score(y_test, preds)

0.5541125541125541

What if we used the old purging method where we remove the rows?

In [65]:
nb = GaussianNB()
nb.fit(x_train, y_train)

preds = nb.predict(x_test)
f1_score(y_test, preds)

0.3687150837988827

So there seems to be a variance here, and I wonder if their results are based on the second purging method (setting them to 1).

What if we set them to 0 (method 3)?

In [77]:
nb = GaussianNB()
nb.fit(x_train, y_train)

preds = nb.predict(x_test)
f1_score(y_test, preds)

0.319634703196347